# LiverNet -- Classification task on KMC dataset

To request access to the KMC dataset, please contact Prof. Shyam Lal ([shyam.mtec@gmail.com](mailto:shyam.mtec@gmail.com)).

In [ ]:
# Importing necessary libraries

## Keras 
from keras.layers import *
from keras.callbacks import *
from keras.optimizers import *
from keras.models import load_model, Model
from keras import backend as K
from keras.utils import conv_utils

## OpenCV
import cv2

## Utility libraries
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

## Tensorflow
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## OPTIONAL: MOUNT GOOGLE DRIVE FILESYS WHEN USING STORING DATA ON GOOGLE DRIVE
# from google.colab import drive
# drive.mount('/content/drive')

## Reading the dataset

### Train - Folds 1,2,3,4
### Test - Fold 5

5-fold cross validation was performed in our training procedure. Here for an example, fold-5 is considered to be the testset. 

In [ ]:
no_of_classes = 4
classes_name = [0,1,2,3]

In [ ]:
train_data = []
train_label = []
test_data = []
test_label = []

## Filepath of root directory of the dataset
src_path = 'ENTER FILEPATH OF ROOT DIR OF DATASET'

## Preparing train data
train_folds = ['fold1','fold2','fold3','fold4']
test_folds = ['fold5']

label=-1

## Extract trainset images for this fold config
for fold_name in train_folds:
  print("inside {}".format(fold_name))
  fold_path = src_path + fold_name + '/'
  for i,can_type in enumerate(sorted(os.listdir(fold_path))):
    label = i
    for img_name in sorted(os.listdir(fold_path + can_type + '/')):
      img = cv2.imread(fold_path + can_type + '/' + img_name,1)
      img = img/255.0
      train_data.append(img)
      train_label.append(label)
      img_hf = cv2.flip(img,0)
      train_data.append(img_hf)
      train_label.append(label)
      img_vf = cv2.flip(img,1)
      train_data.append(img_vf)
      train_label.append(label)

## Extract testset images for this fold config
for fold_name in test_folds:
  print("inside {}".format(fold_name))
  fold_path_test = src_path + fold_name + '/'
  for i,can_type in enumerate(sorted(os.listdir(fold_path_test))):
    label = i
    for img_name in sorted(os.listdir(fold_path_test + can_type + '/')):
      img = cv2.imread(fold_path_test + can_type + '/' + img_name,1)
      img = img/255.0
      test_data.append(img)
      test_label.append(label)

train_label = tf.keras.utils.to_categorical(train_label)
test_label = tf.keras.utils.to_categorical(test_label)

train_data = np.array(train_data)
test_data = np.array(test_data)
train_label = np.array(train_label)
test_label = np.array(test_label)

print('train data size = {}'.format(train_data.shape))
print('train label size = {}'.format(train_label.shape))
print('test data size = {}'.format(test_data.shape))
print('test label size = {}'.format(test_label.shape))

train_data = tf.convert_to_tensor(train_data)
test_data = tf.convert_to_tensor(test_data)
train_label = tf.convert_to_tensor(train_label)
test_label = tf.convert_to_tensor(test_label)                

## Comparison of competitive models

## 1. ResNet50

In [ ]:
def Resnet_transfer_orig():
  base_model_res = tf.keras.applications.ResNet50(weights = None , include_top = False , input_shape=(224,224,3))
  x = base_model_res.output
  x = tf.keras.layers.GlobalAveragePooling2D()(x)
  pred = tf.keras.layers.Dense(no_of_classes, activation='softmax')(x)
  model = tf.keras.models.Model(inputs = base_model_res.input , outputs = pred)
  for layer in model.layers[0:]:
    layer.trainable = True
  
  #model = tf.keras.models.Model(inputs = base_model_res.input , outputs = pred)
  return model 


model_Res =  Resnet_transfer_orig()

for layer in model_Res.layers:
    if hasattr(layer, 'kernel_regularizer'):
        layer.kernel_regularizer= regularizers.l2(0.003)
    
    if hasattr(layer, 'bias_regularizer'):
        layer.bias_regularizer= regularizers.l2(0.003)
adam = tf.keras.optimizers.Adam(lr = 0.001)
model_Res.compile(optimizer = adam , loss = 'categorical_crossentropy' , metrics = ["acc"])
model_Res.summary()

### Training

In [ ]:

def scheduler(epoch, lr):
   if epoch < 10:
     return lr
   else:
     return lr * tf.math.exp(-0.1)


lrate =  tf.keras.callbacks.LearningRateScheduler(scheduler)


history = model_Res.fit(train_data, train_label, batch_size = 4, verbose=1, 
                        steps_per_epoch = len(train_data)//4, 
                         epochs = 40, callbacks = [lrate])
                                  

### Results and Plots

In [ ]:
plt.plot(history.history['loss'] , label = 'train_loss')
##plt.plot(history.history['val_loss'] , label = 'val_loss')
plt.legend()
plt.xlabel("No. of epochs")
plt.ylabel("Loss(Categorical Crossentropy)")
plt.title("Loss vs. Epoch plot for ResNet50")
plt.show()

plt.plot(history.history['acc'] , label = 'train_acc')
##plt.plot(history.history['val_acc'] , label = 'val_acc')
plt.legend()
plt.xlabel("No. of epochs")
plt.ylabel("Accuracy")
plt.title("Accuracy vs. Epoch plot for ResNet50")
plt.show()

print('original')
orig_class_indices = np.argmax(test_label,axis=1) 
print(orig_class_indices)
print('\n')
print('predicted')
#test_step = test_generator.n//test_generator.batch_size
#test_generator.reset()
pred = model_Res.predict(test_data ,batch_size=1, verbose = 1)
pred_class_indices = np.argmax(pred,axis=1)
print(pred_class_indices)
print('\n')

#print(train_generator.class_indices)
#print('\n')
from sklearn.metrics import precision_score,recall_score,accuracy_score,roc_curve, confusion_matrix, roc_auc_score, auc, f1_score,jaccard_score,classification_report
print('Precision {}'.format(precision_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro')))
print('Recall {}'.format(recall_score(y_true=orig_class_indices, y_pred=pred_class_indices,average='micro')))
print('Accuracy {}'.format(accuracy_score(y_true=orig_class_indices, y_pred=pred_class_indices)))
print('F1 {}'.format(f1_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro')))
print('jaccard {}'.format(jaccard_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro')))
print('confusion_matrix\n {}'.format(confusion_matrix(y_true=orig_class_indices, y_pred=pred_class_indices)))
#print('classification_report\n {}'.format(classification_report(y_true=orig_class_indices, y_pred=pred_class_indices)))
print('\n\n')

met = np.zeros((no_of_classes+1,5))

avg_met = [precision_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro'),
           recall_score(y_true=orig_class_indices, y_pred=pred_class_indices,average='micro'),
           f1_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro'),
           jaccard_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro'),
           accuracy_score(y_true=orig_class_indices, y_pred=pred_class_indices)]

avg_met = np.array(avg_met)
avg_met.round(decimals=2)
met[no_of_classes,:] = avg_met

classes = classes_name


for cl in classes:

    print("class: ",cl)

    a1 = np.uint8(orig_class_indices == cl)
    a2 = np.uint8(pred_class_indices == cl)

    print('Accuracy {}'.format(accuracy_score(y_true=a1, y_pred=a2)))
    print('F1 {}'.format(f1_score(y_true=a1, y_pred=a2)))
    print('precision {}'.format(precision_score(y_true=a1, y_pred=a2)))
    print('recall {}'.format(recall_score(y_true=a1, y_pred=a2)))

    print('jaccard {}'.format(jaccard_score(y_true=a1, y_pred=a2)))
    print("_______________________________")

    class_met = [precision_score(y_true=a1, y_pred=a2),
                 recall_score(y_true=a1, y_pred=a2),
                 f1_score(y_true=a1, y_pred=a2),
                 jaccard_score(y_true=a1, y_pred=a2),
                 accuracy_score(y_true=a1, y_pred=a2)]

    class_met = np.array(class_met)
    class_met.round(decimals=2)
    met[cl,:] =class_met


print(met)
np.save('/content/drive/MyDrive/LiverNet/KMC_Liver_dataset_folds/Metrics/resnet50_liver_kmc_met_5.npy',met)

from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
cm = confusion_matrix(y_true=orig_class_indices, y_pred=pred_class_indices)
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
plot_confusion_matrix(cm, classes_name,
                        normalize=False,
                        title='Confusion matrix for ResNet50',
                        cmap=plt.cm.Blues)




## 2. DenseNet

In [ ]:
def DenseNet_transfer_orig():
  base_model_dense = tf.keras.applications.DenseNet121(weights = None , include_top = False , input_shape=(224,224,3))
  x = base_model_dense.output
  x = tf.keras.layers.GlobalAveragePooling2D()(x)
  pred = tf.keras.layers.Dense(no_of_classes, activation='softmax')(x)
  model = tf.keras.models.Model(inputs = base_model_dense.input , outputs = pred)
  for layer in model.layers[0:]:
    layer.trainable = True
  
  return model 


model_dense =  DenseNet_transfer_orig()

for layer in model_dense.layers:
    if hasattr(layer, 'kernel_regularizer'):
        layer.kernel_regularizer= regularizers.l2(0.003)
    
    if hasattr(layer, 'bias_regularizer'):
        layer.bias_regularizer= regularizers.l2(0.003)
adam = tf.keras.optimizers.Adam(lr = 0.001)
model_dense.compile(optimizer = adam , loss = 'categorical_crossentropy' , metrics = ["acc"])
model_dense.summary()

### Training

In [ ]:

def scheduler(epoch, lr):
   if epoch < 10:
     return lr
   else:
     return lr * tf.math.exp(-0.1)


lrate =  tf.keras.callbacks.LearningRateScheduler(scheduler)


history = model_dense.fit(train_data, train_label, batch_size = 4, verbose=1, 
                         steps_per_epoch = len(train_data)//4, 
                         epochs = 40, callbacks = [lrate]
                          )
                                  

### Results and plots

In [ ]:
plt.plot(history.history['loss'] , label = 'train_loss')
#plt.plot(history.history['val_loss'] , label = 'val_loss')
plt.legend()
plt.xlabel("No. of epochs")
plt.ylabel("Loss(Categorical Crossentropy)")
plt.title("Loss vs. Epoch plot for DenseNet121")
plt.show()

plt.plot(history.history['acc'] , label = 'train_acc')
#plt.plot(history.history['val_acc'] , label = 'val_acc')
plt.legend()
plt.xlabel("No. of epochs")
plt.ylabel("Accuracy")
plt.title("Accuracy vs. Epoch plot for DenseNet121")
plt.show()

print('original')
orig_class_indices = np.argmax(test_label,axis=1) 
print(orig_class_indices)
print('\n')
print('predicted')
#test_step = test_generator.n//test_generator.batch_size
#test_generator.reset()
pred = model_dense.predict(test_data ,batch_size=1, verbose = 1)
pred_class_indices = np.argmax(pred,axis=1)
print(pred_class_indices)
print('\n')

#print(train_generator.class_indices)
#print('\n')
from sklearn.metrics import precision_score,recall_score,accuracy_score,roc_curve, confusion_matrix, roc_auc_score, auc, f1_score,jaccard_score,classification_report
print('Precision {}'.format(precision_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro')))
print('Recall {}'.format(recall_score(y_true=orig_class_indices, y_pred=pred_class_indices,average='micro')))
print('Accuracy {}'.format(accuracy_score(y_true=orig_class_indices, y_pred=pred_class_indices)))
print('F1 {}'.format(f1_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro')))
print('jaccard {}'.format(jaccard_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro')))
print('confusion_matrix\n {}'.format(confusion_matrix(y_true=orig_class_indices, y_pred=pred_class_indices)))
#print('classification_report\n {}'.format(classification_report(y_true=orig_class_indices, y_pred=pred_class_indices)))
print('\n\n')

met = np.zeros((no_of_classes+1,5))

avg_met = [precision_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro'),
           recall_score(y_true=orig_class_indices, y_pred=pred_class_indices,average='micro'),
           f1_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro'),
           jaccard_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro'),
           accuracy_score(y_true=orig_class_indices, y_pred=pred_class_indices)]

avg_met = np.array(avg_met)
avg_met.round(decimals=2)
met[no_of_classes,:] = avg_met

classes = classes_name


for cl in classes:

    print("class: ",cl)

    a1 = np.uint8(orig_class_indices == cl)
    a2 = np.uint8(pred_class_indices == cl)

    print('Accuracy {}'.format(accuracy_score(y_true=a1, y_pred=a2)))
    print('F1 {}'.format(f1_score(y_true=a1, y_pred=a2)))
    print('precision {}'.format(precision_score(y_true=a1, y_pred=a2)))
    print('recall {}'.format(recall_score(y_true=a1, y_pred=a2)))

    print('jaccard {}'.format(jaccard_score(y_true=a1, y_pred=a2)))
    print("_______________________________")

    class_met = [precision_score(y_true=a1, y_pred=a2),
                 recall_score(y_true=a1, y_pred=a2),
                 f1_score(y_true=a1, y_pred=a2),
                 jaccard_score(y_true=a1, y_pred=a2),
                 accuracy_score(y_true=a1, y_pred=a2)]

    class_met = np.array(class_met)
    class_met.round(decimals=2)
    met[cl,:] =class_met


print(met)
np.save('/content/drive/MyDrive/LiverNet/KMC_Liver_dataset_folds/Metrics/densenet121_liver_kmc_met_5.npy',met)

from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
cm = confusion_matrix(y_true=orig_class_indices, y_pred=pred_class_indices)
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
plot_confusion_matrix(cm, classes_name,
                        normalize=False,
                        title='Confusion matrix for DenseNet121',
                        cmap=plt.cm.Blues)




## 3. InceptionResNetV2

In [ ]:
def InceptionResNet_transfer_orig():
  base_model_inres = tf.keras.applications.InceptionResNetV2(weights = None , include_top = False , input_shape=(224,224,3))
  x = base_model_inres.output
  x = tf.keras.layers.GlobalAveragePooling2D()(x)
  pred = tf.keras.layers.Dense(no_of_classes, activation='softmax')(x)
  model = tf.keras.models.Model(inputs = base_model_inres.input , outputs = pred)
  for layer in model.layers[0:]:
    layer.trainable = True
  
  #model = tf.keras.models.Model(inputs = base_model_res.input , outputs = pred)
  return model 


model_inres =  InceptionResNet_transfer_orig()

for layer in model_inres.layers:
    if hasattr(layer, 'kernel_regularizer'):
        layer.kernel_regularizer= regularizers.l2(0.003)
    
    if hasattr(layer, 'bias_regularizer'):
        layer.bias_regularizer= regularizers.l2(0.003)
adam = tf.keras.optimizers.Adam(lr = 0.001)
model_inres.compile(optimizer = adam , loss = 'categorical_crossentropy' , metrics = ["acc"])
model_inres.summary()

### Training

In [ ]:

def scheduler(epoch, lr):
   if epoch < 10:
     return lr
   else:
     return lr * tf.math.exp(-0.1)


lrate =  tf.keras.callbacks.LearningRateScheduler(scheduler)



history = model_inres.fit(train_data, train_label, batch_size = 4, verbose=1, 
                        steps_per_epoch = len(train_data)//4, 
                         epochs = 40,callbacks = [lrate]
                        )
                                  

### Results and plots

In [ ]:
plt.plot(history.history['loss'] , label = 'train_loss')
#plt.plot(history.history['val_loss'] , label = 'val_loss')
plt.legend()
plt.xlabel("No. of epochs")
plt.ylabel("Loss(Categorical Crossentropy)")
plt.title("Loss vs. Epoch plot for InceptionResNetV2")
plt.show()

plt.plot(history.history['acc'] , label = 'train_acc')
#plt.plot(history.history['val_acc'] , label = 'val_acc')
plt.legend()
plt.xlabel("No. of epochs")
plt.ylabel("Accuracy")
plt.title("Accuracy vs. Epoch plot for InceptionResNetV2")
plt.show()

print('original')
orig_class_indices = np.argmax(test_label,axis=1) 
print(orig_class_indices)
print('\n')
print('predicted')
#test_step = test_generator.n//test_generator.batch_size
#test_generator.reset()
pred = model_inres.predict(test_data ,batch_size=1 , verbose = 1)
pred_class_indices = np.argmax(pred,axis=1)
print(pred_class_indices)
print('\n')

#print(train_generator.class_indices)
#print('\n')
from sklearn.metrics import precision_score,recall_score,accuracy_score,roc_curve, confusion_matrix, roc_auc_score, auc, f1_score,jaccard_score,classification_report
print('Precision {}'.format(precision_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro')))
print('Recall {}'.format(recall_score(y_true=orig_class_indices, y_pred=pred_class_indices,average='micro')))
print('Accuracy {}'.format(accuracy_score(y_true=orig_class_indices, y_pred=pred_class_indices)))
print('F1 {}'.format(f1_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro')))
print('jaccard {}'.format(jaccard_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro')))
print('confusion_matrix\n {}'.format(confusion_matrix(y_true=orig_class_indices, y_pred=pred_class_indices)))
#print('classification_report\n {}'.format(classification_report(y_true=orig_class_indices, y_pred=pred_class_indices)))
print('\n\n')

met = np.zeros((no_of_classes+1,5))

avg_met = [precision_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro'),
           recall_score(y_true=orig_class_indices, y_pred=pred_class_indices,average='micro'),
           f1_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro'),
           jaccard_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro'),
           accuracy_score(y_true=orig_class_indices, y_pred=pred_class_indices)]

avg_met = np.array(avg_met)
avg_met.round(decimals=2)
met[no_of_classes,:] = avg_met

classes = classes_name


for cl in classes:

    print("class: ",cl)

    a1 = np.uint8(orig_class_indices == cl)
    a2 = np.uint8(pred_class_indices == cl)

    print('Accuracy {}'.format(accuracy_score(y_true=a1, y_pred=a2)))
    print('F1 {}'.format(f1_score(y_true=a1, y_pred=a2)))
    print('precision {}'.format(precision_score(y_true=a1, y_pred=a2)))
    print('recall {}'.format(recall_score(y_true=a1, y_pred=a2)))

    print('jaccard {}'.format(jaccard_score(y_true=a1, y_pred=a2)))
    print("_______________________________")

    class_met = [precision_score(y_true=a1, y_pred=a2),
                 recall_score(y_true=a1, y_pred=a2),
                 f1_score(y_true=a1, y_pred=a2),
                 jaccard_score(y_true=a1, y_pred=a2),
                 accuracy_score(y_true=a1, y_pred=a2)]

    class_met = np.array(class_met)
    class_met.round(decimals=2)
    met[cl,:] =class_met


print(met)
np.save('/content/drive/MyDrive/LiverNet/KMC_Liver_dataset_folds/Metrics/InceptionResNetV2_liver_kmc_met_5.npy',met)

from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
cm = confusion_matrix(y_true=orig_class_indices, y_pred=pred_class_indices)
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
plot_confusion_matrix(cm, classes_name,
                        normalize=False,
                        title='Confusion matrix for InceptionResNetV2',
                        cmap=plt.cm.Blues)




## 4. BreastNet

In [ ]:
from keras.layers import *
from keras.callbacks import *
from keras.optimizers import *
from keras.models import load_model, Model

import tensorflow as tf


def cbam_block(cbam_feature, ratio=8):
    """Contains the implementation of Convolutional Block Attention Module(CBAM) block.
    As described in https://arxiv.org/abs/1807.06521.
    """
    
    cbam_feature = channel_attention(cbam_feature, ratio)
    cbam_feature = spatial_attention(cbam_feature)
    return cbam_feature

def channel_attention(input_feature, ratio=8):
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    channel = input_feature.shape[channel_axis]
    
    shared_layer_one = Dense(channel//ratio,
                             activation='relu',
                             kernel_initializer='he_normal',
                             use_bias=True,
                             bias_initializer='zeros')
    shared_layer_two = Dense(channel,
                             kernel_initializer='he_normal',
                             use_bias=True,
                             bias_initializer='zeros')
    
    avg_pool = GlobalAveragePooling2D()(input_feature)    
    avg_pool = Reshape((1,1,channel))(avg_pool)
    assert avg_pool.shape[1:] == (1,1,channel)
    avg_pool = shared_layer_one(avg_pool)
    assert avg_pool.shape[1:] == (1,1,channel//ratio)
    avg_pool = shared_layer_two(avg_pool)
    assert avg_pool.shape[1:] == (1,1,channel)
    
    max_pool = GlobalMaxPooling2D()(input_feature)
    max_pool = Reshape((1,1,channel))(max_pool)
    assert max_pool.shape[1:] == (1,1,channel)
    max_pool = shared_layer_one(max_pool)
    assert max_pool.shape[1:] == (1,1,channel//ratio)
    max_pool = shared_layer_two(max_pool)
    assert max_pool.shape[1:] == (1,1,channel)
    
    cbam_feature = Add()([avg_pool,max_pool])
    cbam_feature = Activation('sigmoid')(cbam_feature)

    if K.image_data_format() == "channels_first":
        cbam_feature = Permute((3, 1, 2))(cbam_feature)
    
    return multiply([input_feature, cbam_feature])

def spatial_attention(input_feature):
    kernel_size = 7
    
    if K.image_data_format() == "channels_first":
        channel = input_feature.shape[1]
        cbam_feature = Permute((2,3,1))(input_feature)
    else:
        channel = input_feature.shape[-1]
        cbam_feature = input_feature
    
    avg_pool = Lambda(lambda x: K.mean(x, axis=3, keepdims=True))(cbam_feature)
    assert avg_pool.shape[-1] == 1
    max_pool = Lambda(lambda x: K.max(x, axis=3, keepdims=True))(cbam_feature)
    assert max_pool.shape[-1] == 1
    concat = Concatenate(axis=3)([avg_pool, max_pool])
    assert concat.shape[-1] == 2
    cbam_feature = Conv2D(filters = 1,
                    kernel_size=kernel_size,
                    strides=1,
                    padding='same',
                    activation='sigmoid',
                    kernel_initializer='he_normal',
                    use_bias=False)(concat)	
    assert cbam_feature.shape[-1] == 1
    
    if K.image_data_format() == "channels_first":
        cbam_feature = Permute((3, 1, 2))(cbam_feature)
        
    return multiply([input_feature, cbam_feature])



def residual_block(y, nb_channels, _strides=(1, 1), _project_shortcut=False):
    shortcut = y

    # down-sampling is performed with a stride of 2
    y = Conv2D(nb_channels, kernel_size=(3, 3), strides=_strides, padding='same')(y)
    y = BatchNormalization()(y)
    y = LeakyReLU()(y)

    y = Conv2D(nb_channels, kernel_size=(3, 3), strides=(1, 1), padding='same')(y)
    y = BatchNormalization()(y)

    # identity shortcuts used directly when the input and output are of the same dimensions
    if _project_shortcut or _strides != (1, 1):
        # when the dimensions increase projection shortcut is used to match dimensions (done by 1×1 convolutions)
        # when the shortcuts go across feature maps of two sizes, they are performed with a stride of 2
        shortcut = Conv2D(nb_channels, kernel_size=(1, 1), strides=_strides, padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)

    y = add([shortcut, y])
    y = LeakyReLU()(y)

    return y



def create_model():
    
    dropRate = 0.3
    
    init = Input((224,224,3))
    x = Conv2D(32, (3, 3), activation=None, padding='same')(init) 
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(32, (3, 3), activation=None, padding='same')(x) 
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x1 = MaxPooling2D((2,2))(x)
    
    x = Conv2D(64, (3, 3), activation=None, padding='same')(x1)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = cbam_block(x)
    x = residual_block(x, 64)
    x2 = MaxPooling2D((2,2))(x)
    
    x = Conv2D(128, (3, 3), activation=None, padding='same')(x2)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = cbam_block(x)
    x = residual_block(x, 128)
    x3 = MaxPooling2D((2,2))(x)
    
    ginp1 = UpSampling2D(size=(2, 2), interpolation='bilinear')(x1)
    ginp2 = UpSampling2D(size=(4, 4), interpolation='bilinear')(x2)
    ginp3 = UpSampling2D(size=(8, 8), interpolation='bilinear')(x3)
    
    hypercolumn = Concatenate()([ginp1, ginp2, ginp3]) 
    gap = GlobalAveragePooling2D()(hypercolumn)

    x = Dense(256, activation=None)(gap)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(dropRate)(x)
    
    x = Dense(256, activation=None)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    y = Dense(no_of_classes, activation='softmax')(x)
   
    model = Model(init, y)
    return model


model_breastnet = create_model()
model_breastnet.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ["acc"])
model_breastnet.summary()

### Training

In [ ]:
def scheduler(epoch, lr):
   if epoch < 10:
     return lr
   else:
     return lr * tf.math.exp(-0.1)


lrate =  tf.keras.callbacks.LearningRateScheduler(scheduler)


history = model_breastnet.fit(train_data, train_label, batch_size = 4, verbose=1, 
                        steps_per_epoch = len(train_data)//4, 
                        epochs = 40,callbacks = [lrate]
                       )
                                  

### Results and Plots

In [ ]:
plt.plot(history.history['loss'] , label = 'train_loss')
#plt.plot(history.history['val_loss'] , label = 'val_loss')
plt.legend()
plt.xlabel("No. of epochs")
plt.ylabel("Loss(Categorical Crossentropy)")
plt.title("Loss vs. Epoch plot for BreastNet")
plt.show()

plt.plot(history.history['acc'] , label = 'train_acc')
#plt.plot(history.history['val_acc'] , label = 'val_acc')
plt.legend()
plt.xlabel("No. of epochs")
plt.ylabel("Accuracy")
plt.title("Accuracy vs. Epoch plot for BreastNet")
plt.show()

print('original')
orig_class_indices = np.argmax(test_label,axis=1) 
print(orig_class_indices)
print('\n')
print('predicted')
#test_step = test_generator.n//test_generator.batch_size
#test_generator.reset()
pred = model_breastnet.predict(test_data ,batch_size=1, verbose = 1)
pred_class_indices = np.argmax(pred,axis=1)
print(pred_class_indices)
print('\n')

#print(train_generator.class_indices)
#print('\n')
from sklearn.metrics import precision_score,recall_score,accuracy_score,roc_curve, confusion_matrix, roc_auc_score, auc, f1_score,jaccard_score,classification_report
print('Precision {}'.format(precision_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro')))
print('Recall {}'.format(recall_score(y_true=orig_class_indices, y_pred=pred_class_indices,average='micro')))
print('Accuracy {}'.format(accuracy_score(y_true=orig_class_indices, y_pred=pred_class_indices)))
print('F1 {}'.format(f1_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro')))
print('jaccard {}'.format(jaccard_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro')))
print('confusion_matrix\n {}'.format(confusion_matrix(y_true=orig_class_indices, y_pred=pred_class_indices)))
#print('classification_report\n {}'.format(classification_report(y_true=orig_class_indices, y_pred=pred_class_indices)))
print('\n\n')

met = np.zeros((no_of_classes+1,5))

avg_met = [precision_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro'),
           recall_score(y_true=orig_class_indices, y_pred=pred_class_indices,average='micro'),
           f1_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro'),
           jaccard_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro'),
           accuracy_score(y_true=orig_class_indices, y_pred=pred_class_indices)]

avg_met = np.array(avg_met)
avg_met.round(decimals=2)
met[no_of_classes,:] = avg_met

classes = classes_name


for cl in classes:

    print("class: ",cl)

    a1 = np.uint8(orig_class_indices == cl)
    a2 = np.uint8(pred_class_indices == cl)

    print('Accuracy {}'.format(accuracy_score(y_true=a1, y_pred=a2)))
    print('F1 {}'.format(f1_score(y_true=a1, y_pred=a2)))
    print('precision {}'.format(precision_score(y_true=a1, y_pred=a2)))
    print('recall {}'.format(recall_score(y_true=a1, y_pred=a2)))

    print('jaccard {}'.format(jaccard_score(y_true=a1, y_pred=a2)))
    print("_______________________________")

    class_met = [precision_score(y_true=a1, y_pred=a2),
                 recall_score(y_true=a1, y_pred=a2),
                 f1_score(y_true=a1, y_pred=a2),
                 jaccard_score(y_true=a1, y_pred=a2),
                 accuracy_score(y_true=a1, y_pred=a2)]

    class_met = np.array(class_met)
    class_met.round(decimals=2)
    met[cl,:] =class_met


print(met)
np.save('/content/drive/MyDrive/LiverNet/KMC_Liver_dataset_folds/Metrics/BreastNet_liver_kmc_met_5.npy',met)

from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
cm = confusion_matrix(y_true=orig_class_indices, y_pred=pred_class_indices)
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
plot_confusion_matrix(cm, classes_name,
                        normalize=False,
                        title='Confusion matrix for BreastNet',
                        cmap=plt.cm.Blues)



## 5. IRRCNN

In [ ]:
def RCNN(filedepth, input):
  conv1 = tf.keras.layers.Conv2D(filters=filedepth, kernel_size=(3, 3), strides=(1, 1), padding='same',activation='relu')(input)
  stack2 = tf.keras.layers.BatchNormalization()(conv1)

  RCL = tf.keras.layers.Conv2D(filters=filedepth, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')

  conv2 = RCL(stack2)
  stack3 = tf.keras.layers.Add()([conv1, conv2])
  stack4 = tf.keras.layers.BatchNormalization()(stack3)

  conv3 = tf.keras.layers.Conv2D(filters=filedepth, kernel_size=(3, 3), strides=(1, 1), padding='same',activation='relu', weights=RCL.get_weights())(stack4)
  stack5 =  tf.keras.layers.Add()([conv1, conv3])
  stack6 = tf.keras.layers.BatchNormalization()(stack5)

  conv4 = tf.keras.layers.Conv2D(filters=filedepth, kernel_size=(3, 3), strides=(1, 1), padding='same',activation='relu', weights=RCL.get_weights())(stack6)
  stack7 =  tf.keras.layers.Add()([conv1, conv4])
  stack8 = tf.keras.layers.BatchNormalization()(stack7)
  return stack8


def inception_block(input,fil):
  conv1 = tf.keras.layers.Conv2D(filters = fil/4, kernel_size = (1,1), strides = (1,1), padding='same')(input)
  conv1 = tf.keras.layers.BatchNormalization()(conv1)
  conv1 = tf.keras.layers.Activation('relu')(conv1)

  conv2 = tf.keras.layers.Conv2D(filters = fil/4, kernel_size = (1,1), strides = (1,1), padding='same')(input)
  conv2 = tf.keras.layers.BatchNormalization()(conv2)
  conv2 = tf.keras.layers.Activation('relu')(conv2)
  conv2 = tf.keras.layers.Conv2D(filters = fil/4, kernel_size = (3,3), strides = (1,1), padding='same')(conv2)
  conv2 = tf.keras.layers.BatchNormalization()(conv2)
  conv2 = tf.keras.layers.Activation('relu')(conv2)

  conv3 = tf.keras.layers.Conv2D(filters = fil/4, kernel_size = (1,1), strides = (1,1), padding='same')(input)
  conv3 = tf.keras.layers.BatchNormalization()(conv3)
  conv3 = tf.keras.layers.Activation('relu')(conv3)
  conv3 = tf.keras.layers.Conv2D(filters = fil/4, kernel_size = (5,5), strides = (1,1), padding='same')(conv3)
  conv3 = tf.keras.layers.BatchNormalization()(conv3)
  conv3 = tf.keras.layers.Activation('relu')(conv3)

  conv4 = tf.keras.layers.MaxPooling2D(pool_size = (3,3),strides = (1,1),padding = 'same')(input)
  conv4 = tf.keras.layers.Conv2D(filters = fil/4, kernel_size = [1,1], strides = (1,1), padding='same')(conv4)
  conv4 = tf.keras.layers.BatchNormalization()(conv4)
  conv4 = tf.keras.layers.Activation('relu')(conv4)

  output = tf.keras.layers.Concatenate()([conv1,conv2,conv3,conv4])
  return output


def irrcnn_block(input,fil):
  rec = RCNN(fil/2,input)
  rec = RCNN(fil/2,rec)

  inc = inception_block(input,fil/2)
  inc = inception_block(input,fil/2)

  ircnn = tf.keras.layers.Concatenate()([rec,inc])
  
  conv =   tf.keras.layers.Conv2D(fil, kernel_size=(1, 1), 
                         activation='relu', padding='same', kernel_initializer='he_normal')(input)
  
  out = tf.keras.layers.Add()([ircnn,conv])
  out = tf.keras.layers.LeakyReLU()(out)
  return out



def get_model():
  input = tf.keras.layers.Input(shape=(224, 224, 3))
  conv1 = tf.keras.layers.Conv2D(16, kernel_size=(7, 7), activation='relu', padding='same', kernel_initializer='he_normal')(input)

  block1 = irrcnn_block(conv1, 32)
  block2 = irrcnn_block(block1, 32)
  block3 = irrcnn_block(block2,32)

  pool1 = tf.keras.layers.MaxPooling2D((3,3), (2,2))(block3)
	
  block4 = irrcnn_block(pool1, 64)
  block5 = irrcnn_block(block4, 64)
  block6 = irrcnn_block(block5,64)

  pool2 = tf.keras.layers.MaxPooling2D((3, 3), (2,2))(block6)

  block7 = irrcnn_block(pool2, 128)
  block8 = irrcnn_block(block7, 128)
  block9 = irrcnn_block(block8,128)	

  pool3 = tf.keras.layers.MaxPooling2D((3, 3), (2,2))(block9)

  block10 = irrcnn_block(pool3, 256)
  block11 = irrcnn_block(block10, 256)
  block12 = irrcnn_block(block11,256)	

  global_pool = tf.keras.layers.GlobalAveragePooling2D()(block12)

  output = tf.keras.layers.Dense(no_of_classes, activation='softmax')(global_pool)
	
  model = tf.keras.models.Model(inputs=input, outputs=output)
  return model


model_irrcnn = get_model()
for layer in model_irrcnn.layers:
    if hasattr(layer, 'kernel_regularizer'):
        layer.kernel_regularizer= regularizers.l2(0.003)
    
    if hasattr(layer, 'bias_regularizer'):
        layer.bias_regularizer= regularizers.l2(0.003)
model_irrcnn.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ["acc"])
model_irrcnn.summary()

### Training

In [ ]:

def scheduler(epoch, lr):
   if epoch < 10:
     return lr
   else:
     return lr * tf.math.exp(-0.1)


lrate =  tf.keras.callbacks.LearningRateScheduler(scheduler)

history = model_irrcnn.fit(train_data, train_label, batch_size = 4, verbose=1, 
                         steps_per_epoch = len(train_data)//4, 
                         epochs = 40,callbacks = [lrate]
                        )
                                  

### Results and Plots

In [ ]:
plt.plot(history.history['loss'] , label = 'train_loss')
#plt.plot(history.history['val_loss'] , label = 'val_loss')
plt.legend()
plt.xlabel("No. of epochs")
plt.ylabel("Loss(Categorical Crossentropy)")
plt.title("Loss vs. Epoch plot for IRRCNN")
plt.show()

plt.plot(history.history['acc'] , label = 'train_acc')
#plt.plot(history.history['val_acc'] , label = 'val_acc')
plt.legend()
plt.xlabel("No. of epochs")
plt.ylabel("Accuracy")
plt.title("Accuracy vs. Epoch plot for IRRCNN")
plt.show()

print('original')
orig_class_indices = np.argmax(test_label,axis=1) 
print(orig_class_indices)
print('\n')
print('predicted')
#test_step = test_generator.n//test_generator.batch_size
#test_generator.reset()
pred = model_irrcnn.predict(test_data ,batch_size=1, verbose = 1)
pred_class_indices = np.argmax(pred,axis=1)
print(pred_class_indices)
print('\n')

#print(train_generator.class_indices)
#print('\n')
from sklearn.metrics import precision_score,recall_score,accuracy_score,roc_curve, confusion_matrix, roc_auc_score, auc, f1_score,jaccard_score,classification_report
print('Precision {}'.format(precision_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro')))
print('Recall {}'.format(recall_score(y_true=orig_class_indices, y_pred=pred_class_indices,average='micro')))
print('Accuracy {}'.format(accuracy_score(y_true=orig_class_indices, y_pred=pred_class_indices)))
print('F1 {}'.format(f1_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro')))
print('jaccard {}'.format(jaccard_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro')))
print('confusion_matrix\n {}'.format(confusion_matrix(y_true=orig_class_indices, y_pred=pred_class_indices)))
#print('classification_report\n {}'.format(classification_report(y_true=orig_class_indices, y_pred=pred_class_indices)))
print('\n\n')

met = np.zeros((no_of_classes+1,5))

avg_met = [precision_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro'),
           recall_score(y_true=orig_class_indices, y_pred=pred_class_indices,average='micro'),
           f1_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro'),
           jaccard_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro'),
           accuracy_score(y_true=orig_class_indices, y_pred=pred_class_indices)]

avg_met = np.array(avg_met)
avg_met.round(decimals=2)
met[no_of_classes,:] = avg_met

classes = classes_name


for cl in classes:

    print("class: ",cl)

    a1 = np.uint8(orig_class_indices == cl)
    a2 = np.uint8(pred_class_indices == cl)

    print('Accuracy {}'.format(accuracy_score(y_true=a1, y_pred=a2)))
    print('F1 {}'.format(f1_score(y_true=a1, y_pred=a2)))
    print('precision {}'.format(precision_score(y_true=a1, y_pred=a2)))
    print('recall {}'.format(recall_score(y_true=a1, y_pred=a2)))

    print('jaccard {}'.format(jaccard_score(y_true=a1, y_pred=a2)))
    print("_______________________________")

    class_met = [precision_score(y_true=a1, y_pred=a2),
                 recall_score(y_true=a1, y_pred=a2),
                 f1_score(y_true=a1, y_pred=a2),
                 jaccard_score(y_true=a1, y_pred=a2),
                 accuracy_score(y_true=a1, y_pred=a2)]

    class_met = np.array(class_met)
    class_met.round(decimals=2)
    met[cl,:] =class_met


print(met)
np.save('/content/drive/MyDrive/LiverNet/KMC_Liver_dataset_folds/Metrics/IRRCNN_liver_kmc_met_5.npy',met)

from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
cm = confusion_matrix(y_true=orig_class_indices, y_pred=pred_class_indices)
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
plot_confusion_matrix(cm, classes_name,
                        normalize=False,
                        title='Confusion matrix for BreastNet',
                        cmap=plt.cm.Blues)




# LiverNet -- Proposed model

[Link to paper](https://link.springer.com/article/10.1007/s11548-021-02410-4)

In [ ]:
def aspp(x,num_fil,input_shape,out_stride):

    """
        ASPP Block
        
        Arguments:
        
            x: input feature map to the ASPP block
            input_shape: input shape of the feature map
            out_stride: the output stride
            
        Returns: 
            
            output feature map after processing
    """
    
    b0=Conv2D(num_fil,(1,1),padding="same",use_bias=False)(x)
    b0=BatchNormalization()(b0)
    b0=Activation("relu")(b0)

    b1=DepthwiseConv2D((3,3),dilation_rate=(2,2),padding="same",use_bias=False)(x)
    b1=BatchNormalization()(b1)
    b1=Activation("relu")(b1)
    b1=Conv2D(num_fil,(1,1),padding="same",dilation_rate = (1,1), use_bias=False)(x)
    b1=BatchNormalization()(b1)
    b1=Activation("relu")(b1)

    b2=DepthwiseConv2D((3,3),dilation_rate=(3,3),padding="same",use_bias=False)(x)
    b2=BatchNormalization()(b2)
    b2=Activation("relu")(b2)
    b2=Conv2D(num_fil, (1,1) , padding="same", use_bias=False)(x)
    b2=BatchNormalization()(b2)
    b2=Activation("relu")(b2)	

    b3=DepthwiseConv2D((3,3),dilation_rate=(6,6),padding="same",use_bias=False)(x)
    b3=BatchNormalization()(b3)
    b3=Activation("relu")(b3)
    b3=Conv2D(num_fil, (1,1) , padding="same", use_bias=False)(x)
    b3=BatchNormalization()(b2)
    b3=Activation("relu")(b2)	

    b5=DepthwiseConv2D((3,3),dilation_rate=(8,8),padding="same",use_bias=False)(x)
    b5=BatchNormalization()(b5)
    b5=Activation("relu")(b5)
    b5=Conv2D(num_fil,(1,1),padding="same",use_bias=False)(b5)
    b5=BatchNormalization()(b5)
    b5=Activation("relu")(b5)
    


    out_shape=int(input_shape[0]/out_stride)
    b4=AveragePooling2D(pool_size=(out_shape,out_shape))(x)
    b4=Conv2D(num_fil,(1,1),padding="same",use_bias=False)(b4)
    b4=BatchNormalization()(b4)
    b4=Activation("relu")(b4)
    b4=BilinearUpsampling((out_shape,out_shape))(b4)

    x=Concatenate()([b4,b0,b1,b2,b3,b5])
    
    x = tf.keras.layers.Conv2D(num_fil , kernel_size = (1,1) , padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    return x


def normalize_data_format(value):
    if value is None:
        value = K.image_data_format()
    data_format = value.lower()
    if data_format not in {'channels_first', 'channels_last'}:
        raise ValueError('The `data_format` argument must be one of '
                         '"channels_first", "channels_last". Received: ' +
                         str(value))
    return data_format




def cbam_block(cbam_feature, ratio=8):
   
    """
        CBAM block
        
        Arguments:
        
            cbam_feature: input feature map to CBAM block
            ratio: channel division ratio in channel attention module
            
        Returns:
        
            output feature map after processing in CBAM block
    """

    cbam_feature = channel_attention(cbam_feature, ratio)
    cbam_feature = spatial_attention(cbam_feature)
    return cbam_feature

def channel_attention(input_feature, ratio=8):
    
    """
        Channel attention module
        
        Arguments:
            
            input_feature: input feature map to channel attention module
            ratio: channel reduction ratio
            
        Returns:
        
            the product of the channel attention map and input feature map 
    """
    
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    channel = input_feature.shape[channel_axis]
    
    shared_layer_one = Dense(channel//ratio,
                             activation='relu',
                             kernel_initializer='he_normal',
                             use_bias=True,
                             bias_initializer='zeros')
    shared_layer_two = Dense(channel,
                             kernel_initializer='he_normal',
                             use_bias=True,
                             bias_initializer='zeros')
    
    avg_pool = GlobalAveragePooling2D()(input_feature)    
    avg_pool = Reshape((1,1,channel))(avg_pool)
    assert avg_pool.shape[1:] == (1,1,channel)
    avg_pool = shared_layer_one(avg_pool)
    assert avg_pool.shape[1:] == (1,1,channel//ratio)
    avg_pool = shared_layer_two(avg_pool)
    assert avg_pool.shape[1:] == (1,1,channel)
    
    max_pool = GlobalMaxPooling2D()(input_feature)
    max_pool = Reshape((1,1,channel))(max_pool)
    assert max_pool.shape[1:] == (1,1,channel)
    max_pool = shared_layer_one(max_pool)
    assert max_pool.shape[1:] == (1,1,channel//ratio)
    max_pool = shared_layer_two(max_pool)
    assert max_pool.shape[1:] == (1,1,channel)
    
    cbam_feature = Add()([avg_pool,max_pool])
    cbam_feature = Activation('sigmoid')(cbam_feature)
    cbam_feature = Activation('sigmoid')(cbam_feature)

    if K.image_data_format() == "channels_first":
        cbam_feature = Permute((3, 1, 2))(cbam_feature)
    
    return multiply([input_feature, cbam_feature])

def spatial_attention(input_feature):
    
    """
        Spatial attention module
        
        Arguments:
            
            input_feature: input feature map
            
        Returns:
            
            product of spatial attention map and input feature map
    """
    
    kernel_size = 7
    
    if K.image_data_format() == "channels_first":
        channel = input_feature.shape[1]
        cbam_feature = Permute((2,3,1))(input_feature)
    else:
        channel = input_feature.shape[-1]
        cbam_feature = input_feature
    
    avg_pool = Lambda(lambda x: K.mean(x, axis=3, keepdims=True))(cbam_feature)
    assert avg_pool.shape[-1] == 1
    max_pool = Lambda(lambda x: K.max(x, axis=3, keepdims=True))(cbam_feature)
    assert max_pool.shape[-1] == 1
    concat = Concatenate(axis=3)([avg_pool, max_pool])
    assert concat.shape[-1] == 2
    cbam_feature = Conv2D(filters = 1,
                    kernel_size=kernel_size,
                    strides=1,
                    padding='same',
                    activation='sigmoid',
                    kernel_initializer='he_normal',
                    use_bias=False)(concat)	
    assert cbam_feature.shape[-1] == 1
    
    if K.image_data_format() == "channels_first":
        cbam_feature = Permute((3, 1, 2))(cbam_feature)
        
    return multiply([input_feature, cbam_feature])



def residual_block(y, nb_channels, _strides=(1, 1), _project_shortcut=False):
    
  """
      Residual block (resnet block)
        
      Arguments: 
            
        y: input feature map
        nb_channels: number of channels
        _strides: output strides
        _project_shortcut: shortcut connection
            
      Returns:
        
        output feature map after processing
        
  """

  shortcut = y
  # down-sampling is performed with a stride of 2
  y = Conv2D(nb_channels//4, kernel_size=(3, 3), strides=_strides, padding='same')(y)
  y = BatchNormalization()(y)
  y = LeakyReLU()(y)

  y = Conv2D(nb_channels, kernel_size=(3, 3), strides=(1, 1), padding='same')(y)
  y = BatchNormalization()(y)

  # identity shortcuts used directly when the input and output are of the same dimensions
  if _project_shortcut or _strides != (1, 1):
    # when the dimensions increase projection shortcut is used to match dimensions (done by 1×1 convolutions)
    # when the shortcuts go across feature maps of two sizes, they are performed with a stride of 2
    shortcut = Conv2D(nb_channels, kernel_size=(1, 1), strides=_strides, padding='same')(shortcut)
    shortcut = BatchNormalization()(shortcut)

  y = add([shortcut, y])
  y = LeakyReLU()(y)

  return y


class BilinearUpsampling(Layer):

    """
        Bilinear Upsampling Class
    """
    
    def __init__(self, upsampling=(2, 2), data_format=None, **kwargs):

        """
            Constructor of Bilinear-Upsampling
        """
        
        super(BilinearUpsampling, self).__init__(**kwargs)
        self.data_format = normalize_data_format(data_format)
        self.upsampling = conv_utils.normalize_tuple(upsampling, 2, 'size')
        self.input_spec = InputSpec(ndim=4)

    def compute_output_shape(self, input_shape):
        height = self.upsampling[0] * \
            input_shape[1] if input_shape[1] is not None else None
        width = self.upsampling[1] * \
            input_shape[2] if input_shape[2] is not None else None
        return (input_shape[0],
                height,
                width,
                input_shape[3])

    def call(self, inputs):
        return tf.image.resize(inputs, (int(inputs.shape[1]*self.upsampling[0]),
                                                   int(inputs.shape[2]*self.upsampling[1])))

    def get_config(self):
        config = {'size': self.upsampling,
                  'data_format': self.data_format}
        base_config = super(BilinearUpsampling, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


def create_model():
    
    """
        Method to create the final model
    """
    
    dropRate = 0.3
    
    init = Input((224,224,3))

    '''
    Replaced two 3x3 convolution from 1 7x7 convolution
    '''



    x = Conv2D(32, (3, 3), activation=None, padding='same')(init) 
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(32, (3, 3), activation=None, padding='same')(x) 
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
   

    '''
    Maxpooling for changed to pool of 3 and stride of 2
    inserted padding = same
    '''

    x1 = MaxPooling2D((3,3),(2,2),padding='same')(x)

      
    
    x = Conv2D(64, (3, 3), activation=None, padding='same')(x1)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = cbam_block(x)

    '''
    reduced first filter by 4 in residual block
    '''

    x = residual_block(x, 64)
    x2 = MaxPooling2D((3,3),(2,2), padding='same')(x)
    
    x = Conv2D(128, (3, 3), activation=None, padding='same')(x2)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = cbam_block(x)
    x = residual_block(x, 128)
    x3 = MaxPooling2D((3,3),(2,2),padding='same')(x)
    
    x1_aspp = aspp(x1,32,(112,112),1)
    x2_aspp = aspp(x2,64,(56,56),1)
    x3_aspp = aspp(x3,128,(28,28),1)

    ginp1 = UpSampling2D(size=(2, 2), interpolation='bilinear')(x1_aspp)
    ginp2 = UpSampling2D(size=(4, 4), interpolation='bilinear')(x2_aspp)
    ginp3 = UpSampling2D(size=(8, 8), interpolation='bilinear')(x3_aspp)
    
    hypercolumn = Concatenate()([ginp1, ginp2, ginp3]) 
    gap = GlobalAveragePooling2D()(hypercolumn)

    x = Dense(256, activation=None)(gap)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(dropRate)(x)
    
    x = Dense(256, activation=None)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    y = Dense(no_of_classes, activation='softmax')(x)
    
    model = Model(init, y)
    return model



model = create_model()

for layer in model.layers:
    if hasattr(layer, 'kernel_regularizer'):
        layer.kernel_regularizer= regularizers.l2(0.03)
    
    if hasattr(layer, 'bias_regularizer'):
        layer.bias_regularizer= regularizers.l2(0.03)


model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ["acc"])
model.summary()

### Training

In [ ]:
def scheduler(epoch, lr):
   if epoch < 10:
     return lr
   else:
     return lr * tf.math.exp(-0.1)


lrate =  tf.keras.callbacks.LearningRateScheduler(scheduler)
history = model.fit(train_data, train_label, batch_size = 4, verbose=1, 
                         steps_per_epoch = len(train_data)//4, 
                         epochs = 40,callbacks = [lrate]
                        )

model.save_weights('/content/drive/MyDrive/LiverNet/KMC_Liver_dataset_folds/Metrics/LiverNet_test_fold5_weights.h5')

### Results and plots

In [ ]:
plt.plot(history.history['loss'] , label = 'train_loss')
#plt.plot(history.history['val_loss'] , label = 'val_loss')
plt.legend()
plt.xlabel("No. of epochs")
plt.ylabel("Loss(Categorical Crossentropy)")
plt.title("Loss vs. Epoch plot for LiverNet")
plt.show()

plt.plot(history.history['acc'] , label = 'train_acc')
#plt.plot(history.history['val_acc'] , label = 'val_acc')
plt.legend()
plt.xlabel("No. of epochs")
plt.ylabel("Accuracy")
plt.title("Accuracy vs. Epoch plot for LiverNet")
plt.show()

print('original')
orig_class_indices = np.argmax(test_label,axis=1) 
print(orig_class_indices)
print('\n')
print('predicted')
#test_step = test_generator.n//test_generator.batch_size
#test_generator.reset()
pred = model.predict(test_data ,batch_size=1, verbose = 1)
pred_class_indices = np.argmax(pred,axis=1)
print(pred_class_indices)
print('\n')

#print(train_generator.class_indices)
#print('\n')
from sklearn.metrics import precision_score,recall_score,accuracy_score,roc_curve, confusion_matrix, roc_auc_score, auc, f1_score,jaccard_score,classification_report
print('Precision {}'.format(precision_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro')))
print('Recall {}'.format(recall_score(y_true=orig_class_indices, y_pred=pred_class_indices,average='micro')))
print('Accuracy {}'.format(accuracy_score(y_true=orig_class_indices, y_pred=pred_class_indices)))
print('F1 {}'.format(f1_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro')))
print('jaccard {}'.format(jaccard_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro')))
print('confusion_matrix\n {}'.format(confusion_matrix(y_true=orig_class_indices, y_pred=pred_class_indices)))
#print('classification_report\n {}'.format(classification_report(y_true=orig_class_indices, y_pred=pred_class_indices)))
print('\n\n')

met = np.zeros((no_of_classes+1,5))

avg_met = [precision_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro'),
           recall_score(y_true=orig_class_indices, y_pred=pred_class_indices,average='micro'),
           f1_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro'),
           jaccard_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro'),
           accuracy_score(y_true=orig_class_indices, y_pred=pred_class_indices)]

avg_met = np.array(avg_met)
avg_met.round(decimals=2)
met[no_of_classes,:] = avg_met

classes = classes_name


for cl in classes:

    print("class: ",cl)

    a1 = np.uint8(orig_class_indices == cl)
    a2 = np.uint8(pred_class_indices == cl)

    print('Accuracy {}'.format(accuracy_score(y_true=a1, y_pred=a2)))
    print('F1 {}'.format(f1_score(y_true=a1, y_pred=a2)))
    print('precision {}'.format(precision_score(y_true=a1, y_pred=a2)))
    print('recall {}'.format(recall_score(y_true=a1, y_pred=a2)))

    print('jaccard {}'.format(jaccard_score(y_true=a1, y_pred=a2)))
    print("_______________________________")

    class_met = [precision_score(y_true=a1, y_pred=a2),
                 recall_score(y_true=a1, y_pred=a2),
                 f1_score(y_true=a1, y_pred=a2),
                 jaccard_score(y_true=a1, y_pred=a2),
                 accuracy_score(y_true=a1, y_pred=a2)]

    class_met = np.array(class_met)
    class_met.round(decimals=2)
    met[cl,:] =class_met


print(met)
np.save('/content/drive/MyDrive/LiverNet/KMC_Liver_dataset_folds/Metrics/LiverNet_liver_kmc_met_5.npy',met)

from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
cm = confusion_matrix(y_true=orig_class_indices, y_pred=pred_class_indices)
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
plot_confusion_matrix(cm, classes_name,
                        normalize=False,
                        title='Confusion matrix for IRRCNN',
                        cmap=plt.cm.Blues)




In [ ]:

model.load_weights('/content/drive/MyDrive/LiverNet/KMC_Liver_dataset_folds/Metrics/LiverNet_test_fold5_weights.h5')
print('original')
orig_class_indices = np.argmax(test_label,axis=1) 
print(orig_class_indices)
print('\n')
print('predicted')
#test_step = test_generator.n//test_generator.batch_size
#test_generator.reset()
pred = model.predict(test_data ,batch_size=1, verbose = 1)
pred_class_indices = np.argmax(pred,axis=1)
print(pred_class_indices)
print('\n')

#print(train_generator.class_indices)
#print('\n')
from sklearn.metrics import precision_score,recall_score,accuracy_score,roc_curve, confusion_matrix, roc_auc_score, auc, f1_score,jaccard_score,classification_report
print('Precision {}'.format(precision_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro')))
print('Recall {}'.format(recall_score(y_true=orig_class_indices, y_pred=pred_class_indices,average='micro')))
print('Accuracy {}'.format(accuracy_score(y_true=orig_class_indices, y_pred=pred_class_indices)))
print('F1 {}'.format(f1_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro')))
print('jaccard {}'.format(jaccard_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro')))
print('confusion_matrix\n {}'.format(confusion_matrix(y_true=orig_class_indices, y_pred=pred_class_indices)))
#print('classification_report\n {}'.format(classification_report(y_true=orig_class_indices, y_pred=pred_class_indices)))
print('\n\n')

met = np.zeros((no_of_classes+1,5))

avg_met = [precision_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro'),
           recall_score(y_true=orig_class_indices, y_pred=pred_class_indices,average='micro'),
           f1_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro'),
           jaccard_score(y_true=orig_class_indices, y_pred=pred_class_indices,average = 'micro'),
           accuracy_score(y_true=orig_class_indices, y_pred=pred_class_indices)]

avg_met = np.array(avg_met)
avg_met.round(decimals=2)
met[no_of_classes,:] = avg_met

classes = classes_name


for cl in classes:

    print("class: ",cl)

    a1 = np.uint8(orig_class_indices == cl)
    a2 = np.uint8(pred_class_indices == cl)

    print('Accuracy {}'.format(accuracy_score(y_true=a1, y_pred=a2)))
    print('F1 {}'.format(f1_score(y_true=a1, y_pred=a2)))
    print('precision {}'.format(precision_score(y_true=a1, y_pred=a2)))
    print('recall {}'.format(recall_score(y_true=a1, y_pred=a2)))

    print('jaccard {}'.format(jaccard_score(y_true=a1, y_pred=a2)))
    print("_______________________________")

    class_met = [precision_score(y_true=a1, y_pred=a2),
                 recall_score(y_true=a1, y_pred=a2),
                 f1_score(y_true=a1, y_pred=a2),
                 jaccard_score(y_true=a1, y_pred=a2),
                 accuracy_score(y_true=a1, y_pred=a2)]

    class_met = np.array(class_met)
    class_met.round(decimals=2)
    met[cl,:] =class_met


print(met)
#np.save('/content/drive/MyDrive/LiverNet/KMC_Liver_dataset_folds/Metrics/LiverNet_liver_kmc_met_5.npy',met)

from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
cm = confusion_matrix(y_true=orig_class_indices, y_pred=pred_class_indices)
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
plot_confusion_matrix(cm, classes_name,
                        normalize=False,
                        title='Confusion matrix for LiverNet',
                        cmap=plt.cm.Blues)


